### 표정 분류

https://drive.google.com/file/d/1lpwQNwijBfaSr8knSNHKWu5KUmzYWU9d/view?usp=drive_link

In [5]:
import os
from glob import glob

train_root ='./datasets/face/train/'
test_root = './datasets/face/test/'

directories = glob(os.path.join(train_root, '*'))

directory_names = []

for directory in directories:
    directory_names.append(directory[directory.rindex('\\')+1:])
print(directory_names)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [7]:
for name in directory_names:
    for i, file_name in enumerate(os.listdir(os.path.join(train_root, name))):
        old_file = os.path.join(train_root + name + '/', file_name)
        new_file = os.path.join(train_root + name + '/', name + str(i + 1) + '.png')

        os.rename(old_file, new_file)

for name in directory_names:
    for i, file_name in enumerate(os.listdir(os.path.join(test_root, name))):
        old_file = os.path.join(test_root + name + '/', file_name)
        new_file = os.path.join(test_root + name + '/', name + str(i + 1) + '.png')

        os.rename(old_file, new_file)

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_data_generator = ImageDataGenerator(rescale=1./255)

train_generator = image_data_generator.flow_from_directory(train_root, target_size=(150, 150), batch_size=32, class_mode='categorical')
print(train_generator.class_indices)

test_generator = image_data_generator.flow_from_directory(test_root, target_size=(150, 150), batch_size=32, class_mode='categorical')
print(test_generator.class_indices)

Found 28709 images belonging to 7 classes.
{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
Found 7178 images belonging to 7 classes.
{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [10]:
import pandas as pd

train_df = pd.DataFrame({'file_paths': train_generator.filepaths, 'targets': train_generator.classes})
test_df = pd.DataFrame({'file_paths': test_generator.filepaths, 'targets': test_generator.classes})

display(train_df.head())
display(test_df.head())

,file_paths,targets
0,./datasets/face/train/angry\angry1.png,0
1,./datasets/face/train/angry\angry10.png,0
2,./datasets/face/train/angry\angry100.png,0
3,./datasets/face/train/angry\angry1000.png,0
4,./datasets/face/train/angry\angry1001.png,0


,file_paths,targets
0,./datasets/face/test/angry\angry1.png,0
1,./datasets/face/test/angry\angry10.png,0
2,./datasets/face/test/angry\angry100.png,0
3,./datasets/face/test/angry\angry101.png,0
4,./datasets/face/test/angry\angry102.png,0


In [11]:
train_df.loc[:, 'file_paths'] = train_df.file_paths.apply(lambda x: x.replace('\\', '/'))
test_df.loc[:, 'file_paths'] = test_df.file_paths.apply(lambda x: x.replace('\\', '/'))

display(train_df.head())
display(test_df.head())

,file_paths,targets
0,./datasets/face/train/angry/angry1.png,0
1,./datasets/face/train/angry/angry10.png,0
2,./datasets/face/train/angry/angry100.png,0
3,./datasets/face/train/angry/angry1000.png,0
4,./datasets/face/train/angry/angry1001.png,0


,file_paths,targets
0,./datasets/face/test/angry/angry1.png,0
1,./datasets/face/test/angry/angry10.png,0
2,./datasets/face/test/angry/angry100.png,0
3,./datasets/face/test/angry/angry101.png,0
4,./datasets/face/test/angry/angry102.png,0


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_df.file_paths, train_df.targets, stratify=train_df.targets, test_size=0.2, random_state=124)

print(y_train.value_counts())
print(y_val.value_counts())

targets
3    5772
4    3972
5    3864
2    3277
0    3196
6    2537
1     349
Name: count, dtype: int64
targets
3    1443
4     993
5     966
2     820
0     799
6     634
1      87
Name: count, dtype: int64


In [14]:
import shutil

root = './datasets/face/'

for file_path in X_val:
    # print(file_path)
    face_dir = file_path[len(root + 'train/'): file_path.rindex('/')]
    destination = os.path.join(root, 'validation/' + face_dir)

    if not os.path.exists(destination):
        os.makedirs(destination)

    shutil.copy2(file_path, destination)
    os.remove(file_path)

In [6]:
import tensorflow as tf

# GPU가 인식되는지 확인
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [9]:
!nvidia-smi

Wed May 29 16:52:46 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.85                 Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   67C    P0             16W /   45W |     295MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----